<a href="https://colab.research.google.com/github/Konic-NLP/similarity-metrics/blob/main/AIDA_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import requests
import json
import re
from re import match
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from collections import defaultdict
from itertools import product
from openpyxl  import load_workbook
from urllib.parse import urlsplit

# run step by step

# helper function and utils


In [3]:
def openwb(workbook):
  wb=load_workbook(workbook)
  sheet=wb.active
  return sheet

In [4]:
def load_json_files(system_json, reference_json):
  with open(system_json) as f:
    system=json.load(f)
  with open(reference_json) as f:
    reference=json.load(f)
  # kairos_ce=openwb('/content/drive/MyDrive/AIDA KAIROS-file code/kairos_ep_alignment_output.tab_sijia_0520.xlsx')
  # template_info= defaultdict(list)
  # for row in list(kairos_ce)[1:]:
  #   template_info[row[3].value]=row[4].value
  return system, reference


In [5]:
def get_template_info(ep_files):
  kairos_ce=openwb(ep_files)
  template_info= defaultdict(list)
  for row in list(kairos_ce)[1:]:
    template_info[row[3].value]=row[4].value
  return template_info

In [6]:
def gold_label(EVAL_ep_file):
  wb=openwb(EVAL_ep_file)

  EVAL_ep=[]
  EVAL_reference_ep=[]
  for row in list(wb)[1:]:
    EVAL_ep.append(row[4].value.strip())
    EVAL_reference_ep.append(row[6].value.strip())
  return EVAL_ep,EVAL_reference_ep

In [7]:
def get_arg_list(arguments_file):
  #return argument list
  ce_arguments=openwb(arguments_file)
  ref_arguments_value=dict()
  ref_arguments_list=[]
  for row in list(ce_arguments)[1:]:

    ref_arguments_value[row[3].value]=row[2].value
    ref_arguments_list.append(row[2].value.strip())
  return arguments_value,arguments_list

In [8]:
def qnode_mapping(arguments_list):

  for v in ref_arguments_list:
    try:
      data_dict=map_qnode(v)
      qnode_=[urlsplit(k['item']['value']).path.split('/')[-1] for k in data_dict]
      if qnode_==[]:
        length=len(v.split())
        # print(length)
        start=1
        while start<length and qnode_==[]:
          # print(' '.join(v.split()[start:]))
          data_dict=map_qnode(' '.join(v.split()[start:]))
          qnode_=[urlsplit(k['item']['value']).path.split('/')[-1] for k in data_dict]
          start+=1

      print([urlsplit(k['item']['value']).path.split('/')[-1] for k in data_dict ])
    except:
      print(k,v)

In [9]:
def extract_filler(system):# system:
 # input system json files
# this is for those don't have corresponding arguments info in the arguments files, we extract the argument value from the existing template.
  dwd2template=defaultdict()
  for k, v in system.items():
    arg2num=dict()  # store the arguments corresponding arg_num
    if system[k]['arguments']!=[]: # if exists arguments in system

      for arg in system[k]['arguments']: # iterate arguments
        if arg['description'] not in arg2num:  # avoid the duplicate 
          arg2num[arg['description']]=arg['arg_num']  # {sydney airport:A3_gol_destination_destination}
      # print(arg2num)
      # print(system[k]['template_info'])
      info=system[k]['template_info'] # get the complete template info here
      pattern=re.compile('<(.*?)>')
      valid=filter(lambda x: re.match('((?!arg\d).)',x),pattern.findall(info)) # get all fillers in the template 
      matched=pattern.findall(info)  # all slots   including arg1(means there is no filler to fill in the arg1 slot)
      c=info
      visited=[]
      for g in range(len(matched)):
        if re.match('((?!arg\d).)',matched[g]):
          
          if '&' in matched[g]:  # 'amer kyhat&Kyhat'
            
            arg_list=matched[g].split('&')
            for candidate in arg_list:
              if candidate.strip() not in visited:
                first_one=''+candidate.strip()
          else:
            first_one=''+matched[g].strip()
          
          # print(matched[g],first_one)
          c=re.sub(matched[g],arg2num[first_one],c,1)
          visited.append(first_one)
      # print(c,k)  
        # else:
        #   print(pattern.findall(info)[g])
      result=pattern.findall(c)
      # print(result)
      # print(len(result), len(set([t.upper() for t in result])))                       
      # if len(result)!=len(set([t.lower() for t in result])): 
      dwd2template[system[k]['dwd_key']]=c
  return dwd2template

In [10]:
# get the final qnode for reference
def get_ref_qnode(reference_arguments_file):#reference_arguments_file:
  ref_arg_newmatch=defaultdict(dict)# initilize a new nested dict
  ref_arg_file=pd.read_excel(reference_arguments_file)  # load the arguments file
  for j,i in enumerate(ref_arg_file['eventprimitive_id']):
    if i != 'EMPTY_NA':  # rules: 1) if we get the mapped qnode based on the arguments value and filtered, we use that; 
    #2)otherwise, we the same as 1) but no filter 
    #3) otherwise, we use the original qnode based on the entity type
    # but there might be one slot has multiple arguments like  Khayat and Amer Khayat both consitutes ARG1, so it's a list
      if ref_arg_file.iloc[j,-1].strip()!='':
        # print(eval(ref_arg_file.iloc[j,-1]))
        map_value=list(eval(ref_arg_file.iloc[j,-1]))
        
      elif  ref_arg_file.iloc[j,-2].strip()!='[]':
        # mapping_value=ref
        map_value=list(eval(ref_arg_file['value_mapping'][j]))
      else:
        map_value= [ref_arg_file['dwd_key'][j].strip('DWD_')]
      ref_arg_newmatch[i][ref_arg_file.iloc[j,-5]]=ref_arg_newmatch[i].get(ref_arg_file.iloc[j,-5],[])+map_value
  return ref_arg_newmatch

In [11]:
# get the final qnode for the system
def get_sys_qnode(system_argumments_file):# system_argument file
  sys_arg_newmatch=defaultdict(dict)
  sys_arg_file=pd.read_excel(system_argumments_file)
  for j,i in enumerate(sys_arg_file['system_ep_id']):
    if i != 'EMPTY_NA':
      if sys_arg_file.iloc[j,-1].strip()!='':
        
        map_value=list(eval(sys_arg_file.iloc[j,-1]))
      elif  sys_arg_file.iloc[j,-2].strip()!="[]":
        # mapping_value=ref
        # print(sys_arg_file.iloc[j,-2].strip())
        map_value=list(eval(sys_arg_file['mapping_value'][j]))
      else:
    
        map_value= [sys_arg_file.iloc[j,-3].strip('DWD_')]
      # if sys_arg_file.iloc[j,-4] in  sys_arg_newmatch[i]:
      sys_arg_newmatch[i][sys_arg_file.iloc[j,-4]]=sys_arg_newmatch[i].get(sys_arg_file.iloc[j,-4],[])+map_value
  return sys_arg_newmatch

In [12]:
def get_candidates(system_args_file,reference_args_file):
  ref_argument_wk=load_workbook('/content/drive/MyDrive/AIDA KAIROS-file code/ce1005_arguments_sijia_0520.tab.xlsx')
  ref_argument_sheet=ref_argument_wk.active
  ce2args=defaultdict(list)
  for row in list(ref_argument_sheet)[1:]:
    if row[0].value.strip() != 'EMPTY_NA':
      # print(row[-2].value)
      ce2args[row[0].value].append(row[-2].value)
  sys_argument_wk=load_workbook('/content/drive/MyDrive/AIDA KAIROS-file code/kairos_ke_analysis_output_sijia_0520.tab.xlsx')
  sys_argument_sheet=sys_argument_wk.active
  kai2args=defaultdict(list)
  for row in list(sys_argument_sheet)[1:]:
    if row[0].value.strip() != 'EMPTY_NA':
    # print(row[-2].value)
      kai2args[row[3].value].append(row[-1].value)
  return ce2args,kai2args

# kgtk SPARQL PART

In [ ]:
def query_parent(qnode):
  # get all the ancestors qnode that has the instance-of or subclass-of with the query qnode
  url = 'https://query.wikidata.org/sparql'
  query = '''
SELECT ?ancestor WHERE{
wd:%s wdt:P31+|wdt:P279+ ?ancestor }

  '''%(qnode)
  try:
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}
    r = requests.get(url, params = {'format': 'json', 'query': query},headers=headers)
    
    data = r.json()
  except:
    print(r)
  return [c['ancestor']['value'].strip('http://www.wikidata.org/entity/') for c in data['results']['bindings']]

In [13]:
def call_semantic_similarity(input_file, url='https://kgtk.isi.edu/similarity_api'):
  #  get the similarity score by batch query
  file_name = os.path.basename(input_file)
  files = {
      'file': (file_name, open(input_file, mode='rb'), 'application/octet-stream')
  }
  resp = requests.post(url, files=files, params={'similarity_types': 'all'})
  if resp.status_code!=500:
  

    s = json.loads(resp.json())
    return pd.DataFrame(s)
    print(s)
     

In [ ]:
def map_qnode(query_items):
  #map the qnode based on the argument value,
  url = 'https://query.wikidata.org/sparql'
  query = '''
  SELECT ?item ?prefLabel WHERE {
          VALUES ?prefLabel { "%s"@en }
          ?item rdfs:label ?prefLabel 

  }

  '''%(query_items)
  try:
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}
    r = requests.get(url, params = {'format': 'json', 'query': query},headers=headers)
    
    data = r.json()
  except:
    print(r)
  return data['results']['bindings']

# lexical overlapping score

In [14]:
def get_arg_match(system_event):  # get the arguments in t
  arg_match=set()
  # for k in system.keys():
  sys_event=system[system_event]
  # if sys_event['arguments']==[]:
  template_info=sys_event['template_info']
  pattern=re.compile('<(.*?)>')
  # valid=filter(lambda x: re.match('((?!arg\d).)',x),pattern.findall(info))
  matched=pattern.findall(template_info)
  if sys_event['dwd_key'] in dwd2template.keys():
    tem_matched=pattern.findall(dwd2template[sys_event['dwd_key']])
    # print(tem_matched,matched)
    assert len(matched)==len(tem_matched)
    for c in zip(tem_matched,matched):
      # print(c)
      if '&' in c[1]:
        for g in c[1].split('&'):
          arg_match.add((c[0].upper(),g.strip()))
      else:    
        arg_match.add((c[0].upper(),c[1].strip()))
  print('sys',sorted(arg_match))
  return [c for c in arg_match if not c[1].startswith('arg')]

In [15]:
def get_ref_match(reference_):
  arg_match=set()
  
  reference_event=reference[reference_]
  for k in reference_event['arguments']:
    arg_match.add((k['arg_num'].upper(),k['description']))
  print('ref',sorted(arg_match))
  return arg_match

In [16]:
def get_score(q1,q2):
  score=0
  arg_visited=defaultdict(float)
  argmatch=get_arg_match(q1)
  argmatch2=get_ref_match(q2)
  # print(argmatch,argmatch2)
  for c in argmatch:
    for g in argmatch2:

      if c[0]==g[0]:# ensure these two are the same  argument
        _pattern = re.compile(r"'s")
        c_args=[ re.sub(_pattern,'',arg.lower()) for arg in     c[1].split() if not re.match('arg\d',c[1])]
        g_args=[ re.sub(_pattern,'',arg.lower()) for arg in     g[1].split() if not re.match('arg\d',g[1])]
        print(c_args, g_args)
        num_overlap=len(set(c_args)&set(g_args))*2
        num_total= len(set(c_args))+len(set(g_args))
        # print(num_overlap,num_total)
        if num_total==0:
          return 0
        else:
          score+= num_overlap/num_total
        # if score >= arg_visited[c[0]]:
          # arg_visited[c[0]]=score
      
  # if len(arg_visited)==0:
    # return 0
  # else:
    # return sum([t for s,t in arg_visited.items()])
  # return score/len([t for t in argmatch2 if not re.match('arg\d',t[1])]),sorted(argmatc
  # h),sorted(argmatch2)
  return score

# embedding score with sentence-transformers


In [17]:
model = SentenceTransformer('all-MiniLM-L6-v2')  # get the model

In [18]:
def get_embedding_score(q1,q2):  # compute the embedding_score
  score=0
  argmatch=get_arg_match(q1)
  argmatch2=get_ref_match(q2)
  # print(argmatch,argmatch2)
  arg_visited=defaultdict(float)
  for c in argmatch:
    for g in argmatch2:
      
      if c[0]==g[0]:# ensure these two are the same  argument
        if not re.match('arg\d',c[1]) and  not re.match('arg\d',g[1]):
          # print(c[1],g[1])
          sys_arg=model.encode(c[1])
          ref_arg=model.encode(g[1])
          result=np.dot(sys_arg,ref_arg)
          # if result >= arg_visited[c[0]]:
          #   arg_visited[c[0]]=result
          # else:
          #   result=arg_visited[c[0]]
          if result>=0.5:
            score0=result
          else:
            score0=0
          # print(result)
    
          score+= score0

  
      
  
  # print(len([t for t in argmatch2 if not re.match('arg\d',t[1])]))
  # return score/len([t for t in argmatch2 if not re.match('arg\d',t[1])]),sorted(argmatch),sorted(argmatch2)
  return score
  # if len(arg_visited)==0:
  #   return 0
  # else:
  #   return sum([v if v>=0.5 else 0 for k,v in arg_visited.items() ])/len(arg_visited)

# kgtk score 

In [19]:
def get_similarity(q1,q2):
  num_=0
  total_score=0
  if q1 in sys_arg_newmatch and q2 in ref_arg_newmatch:
    sys_args=sys_arg_newmatch[q1].keys()
    ref_args=ref_arg_newmatch[q2].keys()
    # get all arguments 
    num_=(len(set([i for i in sys_args if sys_arg_newmatch[i]!=[]])&set([j for j in ref_args if ref_arg_newmatch[q2][j]!=[] ])))
    for sys_arg in sys_args:
      if sys_arg in ref_args:
        best_score=0
        # qnode list
        sys_arg_qnodes=sys_arg_newmatch[q1][sys_arg]
        ref_arg_qnodes= ref_arg_newmatch[q2][sys_arg]
        if sys_arg_qnodes==[]:
          continue
        # print(sys_arg_qnodes,ref_arg_qnodes)
        
        pairs=list(product(*(sys_arg_qnodes,ref_arg_qnodes)))
        # print(pairs)
        s1=[a[0] for a in pairs]
        s2=[a[1] for a  in pairs]
        df={'q1':s1,'q2':s2}
        df=pd.DataFrame(df)
      
        df.to_csv('temp.tsv',sep='\t')
        result=call_semantic_similarity('temp.tsv')
        # print(result)
        # result[result=='']=0
        result.to_csv('test_file_similarity.tsv', index=False, sep='\t')
  
        score=result.apply(get_total_score,axis=1)
        # print(score)
        # print(type(score),max(score))
        print('-------end------')
        total_score+=max(score)
    # print(q2, total_score)
  if num_==0:
    return 0
  return total_score/num_

In [20]:
def get_total_score(row):
  score=0
  if row['q1_label']!='' and row['q2_label']!="":
    row[row==""]=0
    num=0
    for type_score in ['complex','topsim','transe','text','class']:
      if row[type_score]!=0:
        num+=1
    score=row['complex']+row['topsim']+row['transe']+row['text']+row['jc']+row['class']
    if num==0:
      return 0
    else:
      score=score/num
      return score

# final computation and evaluation with 3 scores, first please specify the file path

In [35]:
# change the file path here
system_json='/content/drive/MyDrive/AIDA KAIROS-file code/kairos_with_template_ep.json'   # json files for system event
reference_json='/content/drive/MyDrive/AIDA KAIROS-file code/gold_ep.json'                 # json files for reference events
system_source_file='/content/drive/MyDrive/AIDA KAIROS-file code/kairos_ep_alignment_output.tab_sijia_0520.xlsx'    # source excel files for system events
events_assessment_file='/content/drive/MyDrive/AIDA KAIROS-file code/KAIROS_EVAL_ep_alignment.tab.xlsx'  #human assessment files for events matching
system_arguments_file='/content/drive/MyDrive/AIDA KAIROS-file code/kairos_ke_analysis_output_sijia_0520.tab.xlsx' #excel for system argumentss 
reference_arguments_file='/content/drive/MyDrive/AIDA KAIROS-file code/ce1005_arguments_sijia_0520.tab.xlsx'   # excel files for reference arguments

In [25]:
system,reference=load_json_files(system_json,reference_json)

In [39]:
template_info=get_template_info(system_source_file)
EVAL_ep,EVAL_reference_ep=gold_label(events_assessment_file)
dwd2template=extract_filler(system)
sys_arg_newmatch=get_sys_qnode(system_arguments_file)
ref_arg_newmatch=get_ref_qnode(reference_arguments_file)

In [40]:
# main cell to compute the all score, quite slow
score_record=defaultdict(dict)
for k in system.keys():
  system_ep=system[k]
  if k in EVAL_ep :
    arguments_dict=defaultdict(dict)
    for arg in system_ep['arguments']:
      arguments_dict[arg['dwd_argname']]=arg
    for v in reference.keys():
      score=0
      score1=get_score(k,v)
      score0=get_embedding_score(k,v)
      reference_ep=reference[v]
      if reference_ep['arguments']!=[]:
        if system_ep['system_type'].split('.')[0].lower()==reference_ep['type']:
          ref_argument_dict=defaultdict(dict)
          for arg in reference_ep['arguments']:
            ref_argument_dict[arg['dwd_argname']]=arg
          for args in arguments_dict.keys():
            # print(ref_argument_dict[args])
            # print(arguments_dict)
            if ref_argument_dict[args]!={}:
              # print(ref_argument_dict[args])
              # print(arguments_dict[args])
              # if ref_argument_dict[args]['dwd_key']==arguments_dict[args]['dwd_key']  and ref_argument_dict[args]['type']==arguments_dict[args]['entity_type'].lower():
                score=get_similarity(k,v)
                # print(ref_argument_dict[args],arguments_dict[args])
                # if ref_argument_dict[args]['description']==arguments_dict[args]['description']:
                #   score+=1
          print(k,v,score,score1,score0)
          score_record[k][v]=score+score0+score1

Streaming output truncated to the last 5000 lines.
sys [('ARG1', 'arg1'), ('ARG2', 'arg2'), ('ARG3', 'arg3'), ('ARG4', 'arg4'), ('ARG5', 'luggage')]
ref [('ARG1', 'Khaled Khayat'), ('ARG1', 'Mahmoud Khayat'), ('ARG4', 'Lebanon'), ('ARG5', 'Australia')]
['luggage'] ['australia']
sys [('ARG1', 'arg1'), ('ARG2', 'arg2'), ('ARG3', 'arg3'), ('ARG4', 'arg4'), ('ARG5', 'luggage')]
ref [('ARG1', 'Khaled Khayat'), ('ARG1', 'Mahmoud Khayat'), ('ARG4', 'Lebanon'), ('ARG5', 'Australia')]
CMU_CMU_1005_0002_SEP00155 VP1005.1001 0 0.0 0
sys [('ARG1', 'arg1'), ('ARG2', 'arg2'), ('ARG3', 'arg3'), ('ARG4', 'arg4'), ('ARG5', 'luggage')]
ref [('ARG1', "Khaled Khayat and Mahmoud Khayat's relatives"), ('ARG1', 'Tarek Khayat'), ('ARG4', 'Syria')]
sys [('ARG1', 'arg1'), ('ARG2', 'arg2'), ('ARG3', 'arg3'), ('ARG4', 'arg4'), ('ARG5', 'luggage')]
ref [('ARG1', "Khaled Khayat and Mahmoud Khayat's relatives"), ('ARG1', 'Tarek Khayat'), ('ARG4', 'Syria')]
sys [('ARG1', 'arg1'), ('ARG2', 'arg2'), ('ARG3', 'arg3'), (

In [41]:
# count the evaluation result
hits=0
count=0
for k,v in score_record.items():
  max_score=[x for x,y in v.items() if y==max(v.values())  ]
  print('system_event: '+k, 'match to '+system[k]['match'])
  print(max_score,v)
  if system[k]['match']!='n/a':
    count+=1
# print(max_score[0],system[k]['match'],max_score[0]==system[k]['match'])
  # if (system[k]['match']=='n/a' and max(v.values())==0):
  #   hits+=1
  # if len(max_score)==1:
  if max_score[0].strip()==system[k]['match'].strip():
    hits+=1

system_event: CMU_CMU_1005_0001_SEP00001 match to n/a
['VP1005.1002', 'VP1005.1013', 'VP1005.1040'] {'VP1005.1002': 0.0, 'VP1005.1013': 0.0, 'VP1005.1040': 0.0}
system_event: CMU_CMU_1005_0001_SEP00002 match to VP1005.1035
['VP1005.1021'] {'VP1005.1001': 3.1124192191329794, 'VP1005.1007': 2.978562331199646, 'VP1005.1015': 1.595740787188212, 'VP1005.1019': 3.2556208110805986, 'VP1005.1020': 3.2556208110805986, 'VP1005.1021': 4.699558210865704, 'VP1005.1023': 1.8042778073643935, 'VP1005.1032': 1.5308768493243936, 'VP1005.1033': 2.3779699563980103, 'VP1005.1034': 4.264771143957132, 'VP1005.1035': 3.5749867951977308, 'VP1005.1036': 4.109818314895158, 'VP1005.1046': 0.40119298103, 'VP1005.1052': 3.3968324621518455, 'VP1005.1076': 1.3210067711434412}
system_event: CMU_CMU_1005_0001_SEP00003 match to n/a
['VP1005.1016', 'VP1005.1025', 'VP1005.1077'] {'VP1005.1016': 0, 'VP1005.1025': 0, 'VP1005.1077': 0}
system_event: CMU_CMU_1005_0001_SEP00004 match to n/a
['VP1005.1014'] {'VP1005.1014': 0.33

In [42]:
print('total system events is {} and we predict correct is {}, the total matched system events is {} '.format(len(score_record),hits,count))

total system events is 190 and we predict correct is 15, the total matched system events is 49 
